In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class ACO:
    def __init__(self, num_pop, graph):
        n = len(graph)

        self.population = [[0]] * num_pop # Each ant starts at city number 0
        self.dist = np.zeros(shape=(num_pop)) # dist[ant] = distance made so far

        self.pheromon = np.ones((n, n)) # Ones, so that alpha > 1 will increase probability 

        self.adjacency_matrix = np.zeros(shape=(n, n), dtype=np.float32) # [i, j] = Euclidean distance between city i and j

        # Setting up the distances
        for i in range(n):
            self.adjacency_matrix[i] = np.apply_along_axis(np.linalg.norm, 1, graph[i] - graph)

        # Banning paths from city to itself 
        self.adjacency_matrix = np.where(self.adjacency_matrix == 0, np.inf, self.adjacency_matrix)

    def main_loop(self, num_iter, alpha, beta, Q, rho):
        n = self.adjacency_matrix.shape[0]
        best_route = None
        min_dist = np.inf
        
        mu = 1 / self.adjacency_matrix

        for i in range(num_iter):
            self.population = [[0]] * len(self.population) # Each ant starts at city number 0
            self.dist = np.zeros(self.dist.shape)
            delta_tau = np.zeros_like(self.pheromon) # To perform pheromon decay after loop ends
            
            for ant in range(len(self.population)):
                # Looping until ant hasn't visited all of the cities
                while len(self.population[ant]) != n:
                    # Getting the last city that ant have visited so far
                    current_city = self.population[ant][-1]

                    # Temporary variable to compute probabilities later
                    tmp = (self.pheromon[current_city] ** alpha) * (mu[current_city] ** beta)

                    # Setting visited cities probabilities to zero
                    for j in range(n):
                        if j in self.population[ant]:
                            tmp[j] = 0

                    # Probabilities itself
                    prob = tmp / tmp.sum()

                    # Cumulative probabilites to choose next city
                    cum_prob = np.cumsum(prob)

                    # Choosing next city while it hasn't been visited
                    next_city = np.searchsorted(cum_prob, np.random.rand())

                    while next_city in self.population[ant]:
                        next_city = np.searchsorted(cum_prob, np.random.rand())

                    self.population[ant].append(next_city)
                    self.dist[ant] += self.adjacency_matrix[current_city][next_city]

                    # Updating pheromon
                    self.pheromon[next_city][current_city] += Q / self.dist[ant]
                    self.pheromon[current_city][next_city] += Q / self.dist[ant]

                    delta_tau[next_city][next_city] += Q / self.dist[ant]
                    delta_tau[current_city][next_city] += Q / self.dist[ant]

                    if len(self.population[ant]) == n:
                        # Connecting the last city with the first
                        self.dist[ant] += self.adjacency_matrix[0][self.population[ant][-1]]

                # There's some bug which I don't know how to fix
                if min_dist > self.dist[ant] and self.dist[ant] != 0:
                    min_dist = np.copy(self.dist[ant])
                    best_route = np.copy(self.population[ant])

            print("Iteration: %i\nMin distance: %.3f\nMin route %s" % (i + 1, min_dist, best_route)) 

            # Pheromone decay
            self.pheromon = (1 - rho) * self.pheromon + delta_tau
        
        return min_dist, best_route

In [3]:
def create_circle(n, radius=1):
    circle = []
    theta = 0
    delta = 2 * np.pi / n

    side_length = 2 * radius * np.sin(delta / 2)

    for _ in range(n):
        x = np.cos(theta)
        y = np.sin(theta)
        circle.append(np.array([x, y]))
        theta += delta


    return np.array(circle), n * side_length

In [9]:
file_path = r'D:\Opera_Download\xqf131.tsp'
def read_tsp_file(filepath):
    tsp = []
    with open(filepath) as file:
        for _ in range(8):
            file.readline()
        
        for line in file.readlines()[:-1]:
            _, x, y = line.split()
            tsp.append(np.array([np.float32(x), np.float32(y)]))

    return np.array(tsp)

In [8]:
"""
Min distance: 6.868
Min route [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 18 17 19]
Real ans 6.257
"""
# tsp1 = read_tsp_file(file_path) # Q = 564
graph, perim = create_circle(30)
test = ACO(70, graph)
dist, route = test.main_loop(num_iter=2_000, alpha=1, beta=0.5, rho=0, Q=6)
print("Min dist: %.3f\n Real min dist: %.3f" % (dist, perim))

Iteration: 1
Min distance: 39.957
Min route [ 0 18 22 13  2 20  1  8 12  3 26 27  5 23  7 25 21 19  9 24 11  4  6 14
 15 10 28 17 16 29]
Iteration: 2
Min distance: 34.396
Min route [ 0  4  7  9 16 18 11 10 21 22 20 27 15 14 25 26  8 23  3 24 19 28 29 17
  6  2 13  5 12  1]
Iteration: 3
Min distance: 33.300
Min route [ 0  4 14 19 25 22 13 28 20 21  5  9  6 23  2  3  7 11 17 18 10  8 15 12
 29 16 24  1 27 26]
Iteration: 4
Min distance: 28.968
Min route [ 0  4  7 11  6  2  3  5 29  8 27 20 13 15 14 28 25 26  9 16 19 12 18 22
 21 10 17 23 24  1]
Iteration: 5
Min distance: 28.968
Min route [ 0  4  7 11  6  2  3  5 29  8 27 20 13 15 14 28 25 26  9 16 19 12 18 22
 21 10 17 23 24  1]
Iteration: 6
Min distance: 28.968
Min route [ 0  4  7 11  6  2  3  5 29  8 27 20 13 15 14 28 25 26  9 16 19 12 18 22
 21 10 17 23 24  1]
Iteration: 7
Min distance: 28.668
Min route [ 0  4  7 16 17 25 19 13  5 14 12  8  6  2  3  1 24 21 18 22 29 28 27 23
 10 11 20  9 15 26]
Iteration: 8
Min distance: 28.668
Min rou

In [230]:
arr = np.array([0.1, 0, 0, 0.4, 0, 0, 0.5])
cum_arr = np.cumsum(arr)

array([0.1, 0.1, 0.1, 0.5, 0.5, 0.5, 1. ])

In [175]:
((1 - 0.30901699) ** 2 + 0.95105625 ** 2) ** .5

1.1755702917191821